In [7]:
import pandas as pd
import os
import datetime
import numpy as np
import time
import smtplib
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
import dropbox
from PIL import Image 
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import *

## Get List of Files

In [4]:
def listpic():
    client = dropbox.client.DropboxClient('IS-424yqxy8AAAAAAAAVLOUS9urGIH4kCxP_5Q6hxdz-WrhGMYKa-9MjMZrpwMYZ')
    metadata = client.metadata('/')
    files = [content['path'].split('/')[-1] for content in metadata['contents'] if content['is_dir'] == False ]
    
            
    dates = [content['modified'] for content in metadata['contents'] if content['is_dir'] == False]
    files = pd.DataFrame({'file_name' : files})
    #TIME IS ON UTC TIMEZONE
    files['date_mod_d'] = [time.strptime(i, "%a, %d %b %Y %H:%M:%S %z") for i in dates]
    files['date_mod'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]).date() for i in files.index]
    files['hour_mod'] = [datetime.datetime(*files.date_mod_d.loc[i][:6]).time() for i in files.index]
    
    
    tday = datetime.datetime.today().date()
    files = files[files.date_mod == tday]
    files['f_name'] = [i[0:i.find(' ')].lower() for i in files.file_name]
    files['l_name'] = [i[i.find(' '):i.find('_')].lower() for i in files.file_name]
    files['t_file'] = [i[i.find('.')+1:].lower() for i in files.file_name]

    return files   

listpic()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: DeprecationWarning: You are using a deprecated client. Please use the new v2 client located at dropbox.Dropbox.
  from ipykernel import kernelapp as app


,file_name,date_mod_d,date_mod,hour_mod,f_name,l_name,t_file
0,ABA DUENAS_0001.JPG,"(2017, 5, 12, 20, 2, 58, 4, 132, -1)",2017-05-12,20:02:58,aba,duenas,jpg
1,ABA DUENAS_0005.JPG,"(2017, 5, 12, 20, 3, 3, 4, 132, -1)",2017-05-12,20:03:03,aba,duenas,jpg
2,ABA DUENAS_0009.JPG,"(2017, 5, 12, 20, 3, 7, 4, 132, -1)",2017-05-12,20:03:07,aba,duenas,jpg
3,ADRIANA GRAELL WK27_0001.JPG,"(2017, 5, 12, 20, 3, 13, 4, 132, -1)",2017-05-12,20:03:13,adriana,graell wk27,jpg
4,ADRIANA GRAELL WK27_0013.JPG,"(2017, 5, 12, 20, 3, 15, 4, 132, -1)",2017-05-12,20:03:15,adriana,graell wk27,jpg
5,ADRIANA GRAELL WK27_0013.MPG,"(2017, 5, 12, 20, 3, 21, 4, 132, -1)",2017-05-12,20:03:21,adriana,graell wk27,mpg
6,ADRIANA GRAELL WK27_0023.JPG,"(2017, 5, 12, 20, 3, 23, 4, 132, -1)",2017-05-12,20:03:23,adriana,graell wk27,jpg
7,ADRIANA GRAELL WK27_0024.JPG,"(2017, 5, 12, 20, 3, 26, 4, 132, -1)",2017-05-12,20:03:26,adriana,graell wk27,jpg
8,ADRIANA GRAELL WK27_0028.JPG,"(2017, 5, 12, 20, 3, 32, 4, 132, -1)",2017-05-12,20:03:32,adriana,graell wk27,jpg
9,ADRIANA GRAELL WK27_0033.JPG,"(2017, 5, 12, 20, 3, 37, 4, 132, -1)",2017-05-12,20:03:37,adriana,graell wk27,jpg


In [7]:
listpic().tail()

,file_name,f_name,l_name,t_file,date,hour,size,email
1082,MARIA SANTOS WK18CBFF_0011.JPG,maria,santos wk18cbff,jpg,2017-04-25,11:48:04,0.24,NaN
1083,MARIA SANTOS WK18CBFF_0012.JPG,maria,santos wk18cbff,jpg,2017-04-25,11:48:04,0.25,NaN
1084,MARIA SANTOS WK18CBFF_0013.JPG,maria,santos wk18cbff,jpg,2017-04-25,11:48:06,0.41,NaN
1085,MARIA SANTOS WK18CBFF_0014.JPG,maria,santos wk18cbff,jpg,2017-04-25,11:48:06,0.29,NaN
1086,MARIA SANTOS WK18_0000.JPG,maria,santos wk18,jpg,2017-04-25,11:34:10,0.32,NaN


## Get List of Today's Clients (User Input).

In [5]:
def clients():
    clients = pd.read_excel('registro.xlsx')
    clients['f_name'] = [i[0:i.find(" ")].lower() for i in clients.Name]
    clients['l_name'] = [i[i.find(" "):].lower() for i in clients.Name]
    return clients

## Final List With Emails.

In [6]:
def sendPic():
    sendPic = pd.merge(listpic(), clients(), on = ['f_name', 'l_name'], how = 'outer')
    return sendPic
                
sendPic().head()

,file_name,f_name,l_name,t_file,date,hour,size,email,Name,Email
0,ABA DUENAS_0000.JPG,aba,duenas,jpg,2017-04-21,09:53:48,0.12,NaN,Aba Duenas,fapb88ve@gmail.com
1,ABA DUENAS_0001.AVI,aba,duenas,avi,2017-04-21,09:54:36,10.82,NaN,Aba Duenas,fapb88ve@gmail.com
2,ABA DUENAS_0001.JPG,aba,duenas,jpg,2017-04-21,09:54:30,0.12,NaN,Aba Duenas,fapb88ve@gmail.com
3,ABA DUENAS_0002.JPG,aba,duenas,jpg,2017-04-21,09:54:40,0.36,NaN,Aba Duenas,fapb88ve@gmail.com
4,ABA DUENAS_0003.JPG,aba,duenas,jpg,2017-04-21,09:54:40,0.33,NaN,Aba Duenas,fapb88ve@gmail.com


## Time related Functionalities.

In [ ]:
#def time():
#    exit = datetime.datetime.now()
#    td = datetime.timedelta(minutes = 30)
#    while datetime.datetime.now() < 
#        print('early')
        
#    return datetime.datetime.now().hour
#time()

## Email Sending Functionalities

In [ ]:
def SendMail():
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login(fr, 'rabbitrun88ve')
    
    lp = os.listdir(os.getcwd())[1:9]
    
    emails = ['fapb88ve@gmail.com', 'vcalzadillag@gmail.com', 'balestrinisamuel@gmail.com']
    for i in emails:
        msg = MIMEMultipart()
        msg['Subject'] = 'prueba'
        msg['From'] = fr
        msg['To'] = i
        text = MIMEText("test")
        msg.attach(text)
        for k in lp:
            if '.JPG' in k:
                img_data = open(k, 'rb').read()
                image = MIMEImage(img_data, name=os.path.basename(k))
                msg.attach(image)
            else:
                pass
        s.sendmail(fr, i, msg.as_string())

    
    
    s.quit()

## Watermarking

In [6]:
def watermark(x):
    for i in x:
        if 'AVI' in i:
            clip = VideoFileClip(i)
            logo = (ImageClip("pancita4.png")
              .set_duration(clip.duration)
              .resize(height=80) # if you need to resize...
              .margin(right=8, top=8, opacity=0) # (optional) logo-border padding
              .set_pos(("right","bottom")))
            final = CompositeVideoClip([clip, logo])
            final.write_videofile("Pancitas " + i +".mp4")
        if 'JPG' in i:
            plogo = Image.open('pancita4.png')
            plogow, plogoh = plogo.size
            plogo = plogo.resize((int(plogow*.1), int(plogoh*.1)))
            plogow, plogoh = plogo.size
            pic = Image.open(i)
            picw, pich = pic.size
            pic.paste(plogo, (picw - plogow, pich - plogoh), plogo)
            pic.save('Pancita '+ i)
            
        else:
            pass
